This layout is created 07/22/2025, aiming to test optomechanical coupling. Grating Couplers are used for coupling laser to devices.
This layout contains:
1. Waveguide test patterns: different coupler parameters and waveguide length.
2. Test pattern from Sauer, V. T. K., Diao, Z., Freeman, M. R. & Hiebert, W. K. Optical racetrack resonator transduction of nanomechanical cantilevers. Nanotechnology 25, 055202 (2014).
3. resonators with different parameter of doubly clamped beam.

In [1]:
import gdsfactory as gf
from blocks import resonator_with_beam, cross_section_with_sleeves, ring_resonator, cantilever_beam, my_coupler, waveguide_inv_extrude,tmp_merge_deep_etch_mask
from functools import partial
total_width = 10
@gf.cell
def device_with_io(device_spec, coupler_spec, cross_section, pitch, lateral_offset=0):
    c = gf.Component()
    device = c << device_spec()
    coupler_left = c << coupler_spec()
    coupler_right = c << coupler_spec()
    coupler_left.dmirror_x()
    coupler_left.dmovex(-pitch/2)
    coupler_right.dmovex(pitch/2)
    
    
    ports_distance = abs(device.ports["o1"].x - device.ports["o2"].x)
    
    # center the o1 and o2 ports of the couplers
    center_x = (device.ports["o1"].x + device.ports["o2"].x) / 2
    center_y = (device.ports["o1"].y + device.ports["o2"].y) / 2
    device.move((center_x, center_y),(0, 0))
    
    device.movey(lateral_offset)
    
    radius = cross_section.radius
    device.ports["o1"]
    steps_left = [
        {"x":-radius-ports_distance/2, "y":0},
        {"x":-radius-ports_distance/2, "y":lateral_offset},
    ]
    steps_right = [
        {"x":radius+ports_distance/2, "y":0},
        {"x":radius+ports_distance/2, "y":lateral_offset},
    ]
    
    route_left = gf.routing.route_single(c, coupler_left.ports["o1"], device.ports["o1"],cross_section=cross_section,steps=steps_left if lateral_offset != 0 else None)
    route_right = gf.routing.route_single(c, coupler_right.ports["o1"], device.ports["o2"],cross_section=cross_section,steps=steps_right if lateral_offset != 0 else None)
    return c   
cs = cross_section_with_sleeves(core_width=0.43, total_width=total_width, radius=20)
cs_resonator = cross_section_with_sleeves(core_width=0.43, total_width=total_width, radius=1)
coupler_spec = partial(gf.components.grating_coupler_elliptical, wavelength=1.55,nclad=1,fiber_angle=10, n_periods=60)
gc_spec = partial(my_coupler,coupler=coupler_spec)

wg_spec = partial(waveguide_inv_extrude, width=0.43, length=50, total_width=total_width, sleeve_layer="DEEP_ETCH")


2025-07-23 23:48:25.047 | INFO     | kfactory.kcell:show:3954 - klive v0.3.3: Reloaded file 'c:\Users\weixi\Documents\Python Scripts\NOEMS_Layout\build\gds\beams_cantilever_beam.oas'
2025-07-23 23:48:25.200 | INFO     | kfactory.kcell:show:3954 - klive v0.3.3: Reloaded file 'c:\Users\weixi\Documents\Python Scripts\NOEMS_Layout\build\gds\842210480.oas'
2025-07-23 23:51:13.707 | WARNING  | kfactory.routing.optical:place90:1413 - place90 is deprecated, please use kfactory.routing.optical.place_manhattan instead. place90 will be removed in kfactory 2.0.
2025-07-23 23:51:13.717 | WARNING  | kfactory.routing.optical:place90:1413 - place90 is deprecated, please use kfactory.routing.optical.place_manhattan instead. place90 will be removed in kfactory 2.0.
2025-07-23 23:51:13.916 | INFO     | kfactory.kcell:show:3948 - klive v0.3.3: Opened file 'c:\Users\weixi\Documents\Python Scripts\NOEMS_Layout\build\gds\1949073499.oas'


In [2]:

@gf.cell
def device_cantilever_beam_with_resonator(beam_length=10, beam_width=0.13, beam_gap=0.1,
                              resonator_lengthx=5, resonator_radius=10, resoantor_gap=0.1):
    resonator_spec = partial(
        ring_resonator,
        radius=resonator_radius,
        length_y=0,
        length_x=resonator_lengthx,
        gap=resoantor_gap,
        cross_section=cs_resonator,
        total_width=total_width,
        bend="bend_circular",
        x_span = 10
    )
    cantilever_spec = partial(cantilever_beam, length=beam_length, width=beam_width)

    resonator_with_beam_spec = partial(resonator_with_beam, resonator_spec=resonator_spec, beam_spec=cantilever_spec, gap=beam_gap)

    c = device_with_io(resonator_with_beam_spec, gc_spec, cross_section=cs, pitch=300).copy()
    c.flatten()
    return c
c = device_cantilever_beam_with_resonator(beam_length=15,resonator_radius=10,beam_gap=0.2).copy()
tmp_merge_deep_etch_mask(c)
c.show()


c:\Users\weixi\anaconda3\envs\layout\Lib\site-packages\gdsfactory\components\couplers\coupler90.py:42: UserWarning: {'radius': 10} ignored for cross_section 'xs_f245cf25'
  x = gf.get_cross_section(cross_section, radius=radius)


In [3]:
from blocks import spring_pair, merge_deep_etch_mask, doubly_clamped_beam_with_spring, doubly_clamped_beam, tmp_merge_deep_etch_mask
@gf.cell
def device_ccbeam_with_resonator(beam_length=10, beam_width=0.13, beam_gap=0.1,
                              resonator_lengthx=5, resonator_radius=10, resoantor_gap=0.1):
    resonator_spec = partial(
        ring_resonator,
        radius=resonator_radius,
        length_y=0,
        length_x=resonator_lengthx,
        gap=resoantor_gap,
        cross_section=cs_resonator,
        total_width=total_width,
        bend="bend_circular",
        x_span = 0
    )

    spring_spec = partial(spring_pair, mask_offset=2)
    beam_spec = partial(doubly_clamped_beam, length=beam_length, width=beam_width, mask_offset=2)
    
    ccbeam_spec = partial(doubly_clamped_beam_with_spring,beam_spec=beam_spec, spring_spec=spring_spec)
    
    resonator_with_beam_spec = partial(resonator_with_beam, resonator_spec=resonator_spec, beam_spec=ccbeam_spec, gap=beam_gap)
    c = device_with_io(resonator_with_beam_spec, gc_spec, cross_section=cs, pitch=200,lateral_offset=100).copy()
    c.flatten()
    merge_deep_etch_mask(c)
    return c
c = device_ccbeam_with_resonator(beam_length=100,resonator_radius=5,beam_gap=0.1).copy()
tmp_merge_deep_etch_mask(c)
c.show()


c:\Users\weixi\anaconda3\envs\layout\Lib\site-packages\gdsfactory\components\couplers\coupler90.py:42: UserWarning: {'radius': 5} ignored for cross_section 'xs_f245cf25'
  x = gf.get_cross_section(cross_section, radius=radius)
c:\Users\weixi\anaconda3\envs\layout\Lib\site-packages\kfactory\decorators.py:383: UserWarning: Port type structural not in ['optical', 'electrical', 'placement', 'vertical_te', 'vertical_tm', 'vertical_dual', 'electrical_rf', 'pad', 'pad_rf', 'bump', 'edge_coupler']. Please add it to the port_types list in the config gf.CONF.port_types.
  cell = f(**params)  # type: ignore[call-arg]


In [6]:
from blocks import grating_coupler_test_block
grating_coupler_test_block(gc_spec=gc_spec, cross_section=cs).show()

In [7]:
resonator_spec = partial(
    ring_resonator,
    radius=10,
    length_y=0,
    length_x=10,
    gap=0.2,
    cross_section=cs_resonator,
    total_width=total_width,
    bend="bend_circular",
    x_span = 0
)
c = resonator_spec()
c.show()

In [7]:
import gdsfactory as gf
from functools import partial
from blocks import merge_deep_etch_mask
resonator_spec =  partial(
        ring_resonator,
        radius=10,
        length_y=0,
        length_x=5,
        gap=0.3,
        cross_section=cs_resonator,
        total_width=total_width,
        bend="bend_circular",
        x_span = 0
    )
c = device_with_io(resonator_spec, gc_spec, cross_section=cs, pitch=200,lateral_offset=100).copy()
c.flatten()
merge_deep_etch_mask(c)
c.show()

In [ ]:
from blocks import die_with_alignment_marks
def main():
    c = gf.Component('Main')
    die = die_with_alignment_marks(10000)
    die
die = die_with_alignment_marks(10000)
die.show()